In [110]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import KNNImputer

import pickle

In [111]:
df = pd.read_csv("./data/datos_limpios.csv")

In [112]:
df.head(5)

,KM,Año,Combustible,Ciudad,Nº puertas,Garantía,Cambio,Color,Marca,Modelo,...,Velocidad máxima,Aceleracion,Consumo,Emisiones,Dimensiones,Etiqueta medioambiental,Precio,Largo,Ancho,Alto
0,79996,2019,Diésel,Riba Roja de Turia,5,True,Automático,Azul,SEAT,Leon,...,212,8.7,4.5,117,4548 mm / 1816 mm / 1439 mm,C,16990,4548,1816,1439
1,192300,2009,Diésel,Castuera,3,True,Manual,Beige,Volkswagen,Golf,...,190,10.7,4.5,119,4199 mm / 1779 mm / 1469 mm,B,8800,4199,1779,1469
2,211680,2007,Diésel,Castuera,5,True,Manual,Gris,Toyota,RAV4,...,200,9.3,7.0,185,4395 mm / 1815 mm / 1720 mm,B,9900,4395,1815,1720
3,192549,2011,Diésel,Castuera,5,True,Manual,Gris,Audi,Q5,...,190,11.4,6.5,172,4629 mm / 1880 mm / 1653 mm,B,16900,4629,1880,1653
4,215350,2013,Diésel,Castuera,5,True,Secuencial,Gris,Audi,A7,...,250,5.3,6.4,169,4969 mm / 1911 mm / 1420 mm,B,26500,4969,1911,1420


# Procesado de los datos para el posterior uso en machine learning

Vamos a ir repasando columna por columna viendo que esté todo bien y haciendo las transformaciones necesarias
para poder utilizarla como entrada para nuestros modelos 

### Columnas numéricas

In [113]:
#La columna KM es de tipo numérico y no puede contener valores negativos ya que no tienen sentido
#como si que puede tener valor 0  y dado que todos los vehículos con KM en negativo son del 2024,
#es lógico suponer que tienen 0 KM
df["KM"].describe()

count    5.206600e+04
mean     6.630951e+04
std      6.669503e+04
min     -2.000000e+03
25%      1.950000e+04
50%      5.290250e+04
75%      9.306050e+04
max      2.010810e+06
Name: KM, dtype: float64

In [114]:
df["KM"] = df["KM"].apply(lambda x : x if x>0 else 0)

#Como hemos visto previamente en el EDA, esta columna tiene un par de valores outliers que distorsionan el resto de datos 
#por lo que para ello voy a estar aplicando una transformación logarítmica que suavice los datos más grandes
df["KM_log"] = df["KM"].apply(lambda x: np.log(x + 1))

In [115]:
#Durante el eda también detectamos valores que no tienen sentido en la columna Año
#ya que había coches de más de 1000 años
#En este caso vamos a poner esos valores a none para en un suguiente paso utilizar
#un imputer para determinar los datos que fáltan. 
df["Año"].describe()

count    52066.000000
mean      2008.528022
std        148.521975
min          0.000000
25%       2018.000000
50%       2020.000000
75%       2022.000000
max       2024.000000
Name: Año, dtype: float64

In [116]:
df["Año"] = df["Año"].apply(lambda x : x if x > 900 else np.nan)
print(df["Año"].isna().sum())
df["Año"].describe()

283


count    51783.000000
mean      2019.504490
std          3.550941
min       1996.000000
25%       2018.000000
50%       2020.000000
75%       2022.000000
max       2024.000000
Name: Año, dtype: float64

In [117]:
#En este caso hay que corregir aquellos vehículos con 0 puertas ya que no tiene sentido
#de nuevo las voy a estar poniendo como nan para rellenarlo con el imputer 
df["Nº puertas"] = df["Nº puertas"].apply(lambda x : x if x != 0 else np.nan)

print(df["Nº puertas"].isna().sum())

1973


In [118]:
#Dado que hay muy pocos vehículos con más de 5 puertas voy a estar capando estos a 5 para no confundir al modelo
print(df["Nº puertas"].value_counts())
df["Nº puertas"] = df["Nº puertas"].apply(lambda x : x if x <=5 or np.isnan(x) else 5)

Nº puertas
5.0    43398
4.0     3946
3.0     1712
2.0     1025
6.0        9
8.0        2
7.0        1
Name: count, dtype: int64


In [119]:
#Columna correcta, nada que corregir. Pero tiene todos los valores a true por lo que no vale la pena utilizarla
print(df["Garantía"].isna().sum())
print(df["Garantía"].unique())

df = df.drop(columns=["Garantía"],axis=1)

0
[ True]


In [120]:
#Le faltan algunos datos que están a 0, voy a ponerlos como nan para rellenarlos luego
print(df["Potencia"].describe())
df["Potencia"] = df["Potencia"].apply(lambda x : x if x != 0 else np.nan)

print(df["Potencia"].isna().sum())
df["Potencia"].describe()

count    52066.000000
mean       134.506069
std         66.624357
min          0.000000
25%        100.000000
50%        130.000000
75%        150.000000
max       1020.000000
Name: Potencia, dtype: float64
2018


count    50048.000000
mean       139.929528
std         62.119801
min         30.000000
25%        105.000000
50%        130.000000
75%        150.000000
max       1020.000000
Name: Potencia, dtype: float64

In [121]:
#Le faltan algunos datos que están a 0, voy a ponerlos como nan para rellenarlos luego, también los que estás por debajo de 25 ya que no tiene sentido
print(df["Velocidad máxima"].describe())
df["Velocidad máxima"] = df["Velocidad máxima"].apply(lambda x : x if x > 25 else np.nan)

print(df["Velocidad máxima"].isna().sum())
df["Velocidad máxima"].describe()

count    52066.000000
mean       189.848730
std         30.519817
min          0.000000
25%        178.000000
50%        189.000000
75%        203.000000
max        350.000000
Name: Velocidad máxima, dtype: float64
557


count    51509.000000
mean       191.900348
std         23.410291
min        110.000000
25%        179.000000
50%        189.000000
75%        203.000000
max        350.000000
Name: Velocidad máxima, dtype: float64

In [122]:
#Le faltan algunos datos que están a 0, voy a ponerlos como nan para rellenarlos luego
print(df["Aceleracion"].describe())
df["Aceleracion"] = df["Aceleracion"].apply(lambda x : x if x != 0 else np.nan)

print(df["Aceleracion"].isna().sum())

print(df["Aceleracion"].describe())

count    52066.000000
mean         9.884709
std          3.133130
min          0.000000
25%          8.900000
50%         10.300000
75%         11.500000
max        112.000000
Name: Aceleracion, dtype: float64
2426
count    49640.000000
mean        10.367793
std          2.299488
min          2.100000
25%          9.100000
50%         10.300000
75%         11.500000
max        112.000000
Name: Aceleracion, dtype: float64


In [123]:
#Le faltan algunos datos que están a 0, voy a ponerlos como nan para rellenarlos luego
print(df["Consumo"].describe())
df["Consumo"] = df["Consumo"].apply(lambda x : x if x != 0 else np.nan)

print(df["Consumo"].isna().sum())

print(df["Consumo"].describe())

count    52066.000000
mean         5.663857
std          2.756750
min          0.000000
25%          4.500000
50%          5.200000
75%          6.000000
max         28.300000
Name: Consumo, dtype: float64
273
count    51793.000000
mean         5.693711
std          2.733083
min          0.600000
25%          4.500000
50%          5.200000
75%          6.000000
max         28.300000
Name: Consumo, dtype: float64


In [124]:
#Le faltan algunos datos que están a -1, voy a ponerlos como nan para rellenarlos luego
print(df["Emisiones"].describe())
df["Emisiones"] = df["Emisiones"].apply(lambda x : x if x != -1 else np.nan)

print(df["Emisiones"].isna().sum())

print(df["Emisiones"].describe())

count    52066.000000
mean        86.798832
std         63.957561
min         -1.000000
25%         -1.000000
50%        108.000000
75%        127.000000
max        465.000000
Name: Emisiones, dtype: float64
13515
count    38551.000000
mean       117.578870
std         43.297804
min          0.000000
25%        105.000000
50%        117.000000
75%        135.000000
max        465.000000
Name: Emisiones, dtype: float64


In [125]:
#Esta columna está bien no hay que cambiar nada
print(df["Largo"].describe())

print(df["Largo"].isna().sum())

count    52066.000000
mean      4367.781911
std        319.011282
min       2500.000000
25%       4159.000000
50%       4380.000000
75%       4562.000000
max       5591.000000
Name: Largo, dtype: float64
0


In [126]:
#Estas conlumnas sin embargo tienen algún 0 por lo que lo estaré poniendo como nan
print(df["Ancho"].describe())
df["Ancho"] = df["Ancho"].apply(lambda x : x if x != 0 else np.nan)
print(df["Ancho"].isna().sum())

print(df["Alto"].describe())
df["Alto"] = df["Alto"].apply(lambda x : x if x != 0 else np.nan)
print(df["Alto"].isna().sum())

count    52066.000000
mean      1802.092671
std         80.889874
min          0.000000
25%       1765.000000
50%       1804.000000
75%       1841.000000
max       2107.000000
Name: Ancho, dtype: float64
35
count    52066.000000
mean      1562.907003
std        130.843328
min          0.000000
25%       1461.000000
50%       1535.000000
75%       1639.000000
max       2524.000000
Name: Alto, dtype: float64
20


In [127]:
#Esta columna está bien no hay que cambiar nada, lo único que voy a hacer es eliminar los outliers ya que son muy pocos y aplicar el
#Logaritmo para suavizar los datos
print(df["Precio"].describe())
df = df[df["Precio"] < 500000]
df["Precio_log"] = df["Precio"].apply(lambda x: np.log(x + 1))

df.reset_index(inplace=True, drop=True)

print(df["Precio"].isna().sum())

count    5.206600e+04
mean     2.658286e+04
std      6.577431e+05
min      9.990000e+02
25%      1.499000e+04
50%      1.999000e+04
75%      2.780000e+04
max      1.500000e+08
Name: Precio, dtype: float64
0


### Cadenas de texto o categóricas

In [128]:
#En el caso de la columna ciudad lo más probable es que no la estemos utilizando ya que 
#tiene demasiados valores distios, y en el Eda vimos que realmente no era relevante para el precio
print(df["Ciudad"].nunique())
df = df.drop(columns=["Ciudad"], axis=1)

416


In [129]:
#La columna esta correcta pero necesito dificarla para poder usarla en machine learning
print(df["Combustible"].isna().sum())
print(df["Combustible"].value_counts())

#Vamos a combertirla a una variable numérica. 
#Dado que sus valores no son ordenables voy a usar el get_dummies para transformarla en columnas de 1 y 0s.
columna = "Combustible"

OH_encoder = OneHotEncoder(drop="first")

OH_encoder = OH_encoder.fit(df[columna].values.reshape(-1,1))

columns =  OH_encoder.transform(df[columna].values.reshape(-1,1)).toarray()

pickle.dump(OH_encoder, open(f"./transformadores/{columna}_encoder.pk", 'wb'))


df_OH = pd.DataFrame(data = columns, columns= [columna+"_"+i for i in df[columna].unique()[1:]])

df = pd.concat([df, df_OH], axis = 1)
df_OH.head()

0
Combustible
Gasolina     24072
Diésel       21610
Híbrido       3901
Eléctrico     2216
Otros          263
Name: count, dtype: int64


,Combustible_Gasolina,Combustible_Eléctrico,Combustible_Híbrido,Combustible_Otros
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [130]:
#Esta columna esta bien, no tiene nans pero voy a agrupar los tipo de cambio menos usados en un variable llamada otros
#ya que por separado son muy pocos como para que resulten relevantes
print(df["Cambio"].isna().sum())
print(df["Cambio"].value_counts())

#Tengo que poner el método .strip() porque los valores traen espacios
df["Cambio"] = df["Cambio"].apply(lambda x : x.strip() if x.strip() == "Manual" or x.strip() == "Automático" else "Otro")

print(df["Cambio"].value_counts())

#Vamos a combertirla a una variable numérica. 
#Dado que sus valores no son ordenables voy a usar el get_dummies para transformarla en columnas de 1 y 0s.
columna = "Cambio"

OH_encoder = OneHotEncoder(drop="first")

OH_encoder = OH_encoder.fit(df[columna].values.reshape(-1,1))

columns =  OH_encoder.transform(df[columna].values.reshape(-1,1)).toarray()

pickle.dump(OH_encoder, open(f"./transformadores/{columna}_encoder.pk", 'wb'))


df_OH = pd.DataFrame(data = columns, columns= [columna+"_"+i for i in df[columna].unique()[1:]])


df = pd.concat([df, df_OH], axis = 1)

0
Cambio
Manual              30924
Automático          20877
Secuencial            233
Triptronic             27
No especificado         1
Name: count, dtype: int64
Cambio
Manual        30924
Automático    20877
Otro            261
Name: count, dtype: int64


In [131]:
#Esta columna está correcta no hay que cambiar nada
df["Color"].value_counts()

#Vamos a combertirla a una variable numérica. 
#Dado que sus valores no son ordenables voy a usar el get_dummies para transformarla en columnas de 1 y 0s.
columna = "Color"

OH_encoder = OneHotEncoder(drop="first")

OH_encoder = OH_encoder.fit(df[columna].values.reshape(-1,1))

columns =  OH_encoder.transform(df[columna].values.reshape(-1,1)).toarray()

pickle.dump(OH_encoder, open(f"./transformadores/{columna}_encoder.pk", 'wb'))

df_OH = pd.DataFrame(data = columns, columns= [columna+"_"+i for i in df[columna].unique()[1:]])

df = pd.concat([df, df_OH], axis = 1)

In [132]:
#Esta columna está correcta no hay que cambiar nada que cambiar
print(df["Marca"].nunique())

df["Marca"].value_counts()

#Vamos a combertirla a una variable numérica. 
#En este caso tampoco son ordenables sus valores, pero al tener tantos distintos
#no quiero añadir 46 columnas más al dataframe por lo que usaré un label encoder.
encoder = LabelEncoder()
encoded = encoder.fit(df["Marca"])

df["Marca_encoded"] = encoded.transform(df["Marca"])

pickle.dump(encoded, open("./transformadores/marca_encoder.pk", 'wb'))

46


In [133]:
#Esta columna está correcta no hay que cambiar nada que cambiar
#es demasiado amplia, demasiados valores distintos, lo mejor sería eliminarla
print(df["Modelo"].nunique())

622


In [134]:
#En esta columna lo único que voy a hacer es juntar los valores de "comercial grande" a "vehículo comercial" ya que hay muy poca cantidad
print(df["Tipo de carrocería"].nunique())
df["Tipo de carrocería"] = df["Tipo de carrocería"].apply(lambda x : x if x != "Comercial grande" else "Vehículo comercial")
print(df["Tipo de carrocería"].value_counts())

#Vamos a combertirla a una variable numérica. 
#Dado que sus valores no son ordenables voy a usar el get_dummies para transformarla en columnas de 1 y 0s.
columna = "Tipo de carrocería"

OH_encoder = OneHotEncoder(drop="first")

OH_encoder = OH_encoder.fit(df[columna].values.reshape(-1,1))

columns =  OH_encoder.transform(df[columna].values.reshape(-1,1)).toarray()

pickle.dump(OH_encoder, open(f"./transformadores/Tipo_carroceria_encoder.pk", 'wb'))


df_OH = pd.DataFrame(data = columns, columns= ["Carroceria"+"_"+i for i in df[columna].unique()[1:]])

df = pd.concat([df, df_OH], axis = 1)

9
Tipo de carrocería
Todoterreno           20715
Turismo               18481
Monovolumen            4248
Turismo familiar       3874
Vehículo comercial     3393
Descapotable            691
Coupé                   482
Pick Up                 178
Name: count, dtype: int64


In [135]:
#En esta columna lo único que voy a hacer es juntar los valores de "Cero/ECO" a "Cero" ya que hay muy poca cantidad
print(df["Etiqueta medioambiental"].nunique())
df["Etiqueta medioambiental"] = df["Etiqueta medioambiental"].apply(lambda x : x if x != "Cero/ECO" else "Cero")
print(df["Etiqueta medioambiental"].value_counts())

#Vamos a combertirla a una variable numérica. 
#Dado que sus valores no son ordenables voy a usar el get_dummies para transformarla en columnas de 1 y 0s.
columna = "Etiqueta medioambiental"

OH_encoder = OneHotEncoder(drop="first")

OH_encoder = OH_encoder.fit(df[columna].values.reshape(-1,1))

columns =  OH_encoder.transform(df[columna].values.reshape(-1,1)).toarray()

pickle.dump(OH_encoder, open(f"./transformadores/Etiqueta_encoder.pk", 'wb'))


df_OH = pd.DataFrame(data = columns, columns= ["Etiqueta"+"_"+i for i in df[columna].unique()[1:]])



df = pd.concat([df, df_OH], axis = 1)

6
Etiqueta medioambiental
C               36864
B                6486
Cero             4256
ECO              3509
Sin etiqueta      947
Name: count, dtype: int64


In [136]:
df.shape

(52062, 51)

In [137]:
df.columns

Index(['KM', 'Año', 'Combustible', 'Nº puertas', 'Cambio', 'Color', 'Marca',
       'Modelo', 'Potencia', 'Tipo de carrocería', 'Velocidad máxima',
       'Aceleracion', 'Consumo', 'Emisiones', 'Dimensiones',
       'Etiqueta medioambiental', 'Precio', 'Largo', 'Ancho', 'Alto', 'KM_log',
       'Precio_log', 'Combustible_Gasolina', 'Combustible_Eléctrico',
       'Combustible_Híbrido', 'Combustible_Otros', 'Cambio_Manual',
       'Cambio_Otro', 'Color_Beige', 'Color_Gris', 'Color_Rojo', 'Color_Otro',
       'Color_Amarillo', 'Color_Blanco', 'Color_Naranja', 'Color_Marron',
       'Color_Verde', 'Color_Violeta', 'Color_Negro', 'Marca_encoded',
       'Carroceria_Turismo', 'Carroceria_Todoterreno',
       'Carroceria_Monovolumen', 'Carroceria_Vehículo comercial',
       'Carroceria_Pick Up', 'Carroceria_Descapotable', 'Carroceria_Coupé',
       'Etiqueta_B', 'Etiqueta_Cero', 'Etiqueta_ECO', 'Etiqueta_Sin etiqueta'],
      dtype='object')

In [138]:
df_procesado = df.drop(columns=["KM","Combustible","Cambio","Color","Marca","Modelo","Tipo de carrocería","Dimensiones","Etiqueta medioambiental","Precio"])
df_procesado.head(5)

,Año,Nº puertas,Potencia,Velocidad máxima,Aceleracion,Consumo,Emisiones,Largo,Ancho,Alto,...,Carroceria_Todoterreno,Carroceria_Monovolumen,Carroceria_Vehículo comercial,Carroceria_Pick Up,Carroceria_Descapotable,Carroceria_Coupé,Etiqueta_B,Etiqueta_Cero,Etiqueta_ECO,Etiqueta_Sin etiqueta
0,2019.0,5.0,150.0,212.0,8.7,4.5,117.0,4548,1816.0,1439.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2009.0,3.0,110.0,190.0,10.7,4.5,119.0,4199,1779.0,1469.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007.0,5.0,177.0,200.0,9.3,7.0,185.0,4395,1815.0,1720.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011.0,5.0,143.0,190.0,11.4,6.5,172.0,4629,1880.0,1653.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013.0,5.0,313.0,250.0,5.3,6.4,169.0,4969,1911.0,1420.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Ahora que ya tenemos todos los datos procesados vamos a utilizar el KNNImputer para llenar los NaNs de las filas en la que haya

In [139]:
print(df_procesado["Año"].isna().sum())
print(df_procesado["Aceleracion"].isna().sum())
print(df_procesado["Potencia"].isna().sum())
print(df_procesado["Velocidad máxima"].isna().sum())
print(df_procesado["Consumo"].isna().sum())
print(df_procesado["Emisiones"].isna().sum())

283
2425
2016
557
272
13515


In [140]:
# Inicializamos el objeto Imputer
imputer = KNNImputer(n_neighbors = 3)

# Entrenamos el Imputer y transformamos el DataFrame
transformador = imputer.fit(df_procesado)

datos_procesados = transformador.transform(df_procesado)


pickle.dump(transformador, open("./transformadores/KnnImputer.pk", 'wb'))

df_procesado = pd.DataFrame(data    = datos_procesados,
                  columns = df_procesado.columns)

In [141]:
print(df_procesado["Año"].isna().sum())
print(df_procesado["Aceleracion"].isna().sum())
print(df_procesado["Potencia"].isna().sum())
print(df_procesado["Velocidad máxima"].isna().sum())
print(df_procesado["Consumo"].isna().sum())
print(df_procesado["Emisiones"].isna().sum())

0
0
0
0
0
0


In [142]:
df_procesado.head(5)

,Año,Nº puertas,Potencia,Velocidad máxima,Aceleracion,Consumo,Emisiones,Largo,Ancho,Alto,...,Carroceria_Todoterreno,Carroceria_Monovolumen,Carroceria_Vehículo comercial,Carroceria_Pick Up,Carroceria_Descapotable,Carroceria_Coupé,Etiqueta_B,Etiqueta_Cero,Etiqueta_ECO,Etiqueta_Sin etiqueta
0,2019.0,5.0,150.0,212.0,8.7,4.5,117.0,4548.0,1816.0,1439.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2009.0,3.0,110.0,190.0,10.7,4.5,119.0,4199.0,1779.0,1469.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007.0,5.0,177.0,200.0,9.3,7.0,185.0,4395.0,1815.0,1720.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011.0,5.0,143.0,190.0,11.4,6.5,172.0,4629.0,1880.0,1653.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013.0,5.0,313.0,250.0,5.3,6.4,169.0,4969.0,1911.0,1420.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
df_procesado.to_csv("./data/datos_procesados_2.csv",index = False)
